In [445]:
from __future__ import annotations

import pandas as pd
from typing import Type
import json

In [446]:
materials = pd.read_csv("tables/materials.csv");

In [447]:
materials.head(5)

,id,category,cutting-power,density,cutting-protection,blunt-protection,penentration-protection,magic,name
0,arrow-bone,bow-ammo,1.0,0.2,0.0,0.0,0.0,0.0,Bone arrow
1,arrow-zaz,bow-ammo,2.0,0.2,0.0,0.0,0.0,1.0,Zaz arrow
2,cotton,plant,0.0,0.1,0.2,0.2,0.2,0.1,Cotton
3,textile,material,0.0,1.0,1.0,1.0,1.0,0.1,Textile
4,small-bone-rat,bone,1.0,0.5,0.5,0.5,0.5,0.0,"Bone(rat, small)"


In [448]:
materials.axes

[RangeIndex(start=0, stop=33, step=1),
 Index(['id', 'category', 'cutting-power', 'density', 'cutting-protection',
        'blunt-protection', 'penentration-protection', 'magic', 'name'],
       dtype='object')]

In [449]:
number_columns = ['cutting-power', 'density', 'cutting-protection',
        'blunt-protection', 'penentration-protection', 'magic']
string_columns = ['name']
enum_columns = ['tag', 'category']

## Generating enum

In [450]:
main_string = ""
configuration_string = ""
accessor_string = ""

In [451]:
def enum_name(s: str):
    return s.replace("-", "_").upper()

def method_name(s: str):
    return s.replace("-", "_")

def getter_name(s: str):
    return "get " + method_name(s) + "()"

def new_line(s:str):
    return s + "\n"

def comma(s: str):
    return s + ','

def space(s: str):
    return s + ' '

def quotes(s: str):
    return f'"{s}"'

def tab(s: str):
    return "    " + s

def collection_name(singular: str):
    def _collection_name(s: str):
        return singular + '.' + s
    return _collection_name

def accessor(array_name: str, property_name: str):
    return (
        f'    {getter_name(property_name)} {{\n'
        f'        return MaterialConfiguration.{array_name}[this._id]\n'
        f'    }}\n'
    )

def id_accessor():
    return (
        f'    {getter_name("id")} {{\n'
        f'        return this._id\n'
        f'    }}\n'
    )

In [452]:
class DataEnum:
    def __init__(self, name_singular, name_plural, axis_name) -> None:
        self.name_singular = name_singular
        self.name_plural = name_plural
        self.axis_name = axis_name

    def extract_definition_from_table(self, data: pd.DataFrame):
        return (
            f'export const enum {self.name_singular} ' '{\n'
            f'{data[self.axis_name].apply(enum_name).drop_duplicates(keep="first").apply(comma).apply(new_line).apply(tab).sum()}'
            '}\n'
        )


    def enum_type_symbol(self):
        return f'{enum_name(self.name_singular)}'
    def mapping_type_symbol(self, id_enum:DataEnum):
        return f'Record<{id_enum.name_singular}, {self.name_singular}>'
    def mapping_string_type_symbol(self, id_enum:DataEnum):
        return f'Record<{id_enum.name_singular}, string>'
    def array_type_symbol(self):
        return f'{self.name_singular}[]'
    def array_strings_type_symbol(self):
        return f'Record<{self.enum_type_symbol()}, string>'
    def dict_type_symbol(self):
        return f'Record<string, {self.enum_type_symbol()}>'


    def array_symbol(self):
        return f'{enum_name(self.name_plural)}'
    def array_strings_symbol(self):
        return f'{enum_name(self.name_plural)}_TO_STRING'
    def dict_symbol(self):
        return f'{enum_name(self.name_plural)}_FROM_STRING'


    def mapping_symbol(self, id_enum: DataEnum):
        return f'{enum_name(id_enum.name_singular)}_{enum_name(self.name_singular)}'
    def mapping_string_symbol(self, id_enum: DataEnum):
        return f'{enum_name(id_enum.name_singular)}_{enum_name(self.name_singular)}_STRING'

    def extract_python_list_of_strings(self, data: pd.DataFrame):
        return data[self.axis_name].drop_duplicates(keep="first").values

    def extract_python_list_of_enum_names(self, data: pd.DataFrame):
        return data[self.axis_name].apply(enum_name).drop_duplicates(keep="first").apply(collection_name(self.name_singular)).values

    def extract_array_from_table(self, data: pd.DataFrame):
        return f'    static {self.array_symbol()} : {self.array_type_symbol()} = [{data[self.axis_name].apply(enum_name).drop_duplicates(keep="first").apply(collection_name(self.name_singular)).apply(comma).apply(space).sum()}]\n'
    def extract_string_array_from_table(self, data: pd.DataFrame):
        return f'    static {self.array_strings_symbol()} : {self.array_strings_type_symbol()} = [{data[self.axis_name].apply(quotes).drop_duplicates(keep="first").apply(comma).apply(space).sum()}]\n'


    def extract_inverse_mapping(self, data: pd.DataFrame):
        data_string = ", ".join([f'"{k}": {v}' for k, v in zip(self.extract_python_list_of_strings(data), self.extract_python_list_of_enum_names(data))])
        return f'    static {self.dict_symbol()} : {self.dict_type_symbol()} = {{ {data_string} }}\n'

    def extract_mapping_string(self, data: pd.DataFrame, id_enum: DataEnum):
        return f'    static {self.mapping_string_symbol(id_enum)} : {self.mapping_string_type_symbol(id_enum)} = [{data[self.axis_name].apply(str).apply(quotes).apply(comma).apply(space).sum()}]\n'

    def extract_mapping(self, data: pd.DataFrame, id_enum: DataEnum):
        return f'    static {self.mapping_symbol(id_enum)} : {self.mapping_type_symbol(id_enum)} = [{data[self.axis_name].apply(enum_name).apply(collection_name(self.name_singular)).apply(comma).apply(space).sum()}]\n'

    def getter_string(self, id_enum:DataEnum):
        return accessor(f'{self.mapping_string_symbol(id_enum)}', self.axis_name + "_string")

    def getter(self, id_enum: DataEnum):
        if (id_enum == self):
            return id_accessor()
        return accessor(f'{self.mapping_symbol(id_enum)}', self.axis_name)

In [453]:
enum_MATERIAL_ID = DataEnum("MATERIAL", "MATERIALS", "id")
enum_MATERIAL_CATEGORY = DataEnum("MATERIAL_CATEGORY", "MATERIAL_CATEGORIES", "category")
enums = [enum_MATERIAL_ID, enum_MATERIAL_CATEGORY]

### Enums for categorical data:

In [454]:
for item in enums:
    main_string += item.extract_definition_from_table(materials)

    configuration_string += item.extract_array_from_table(materials)
    configuration_string += item.extract_inverse_mapping(materials)
    configuration_string += item.extract_string_array_from_table(materials)

    configuration_string += item.extract_mapping(materials, enum_MATERIAL_ID)
    configuration_string += item.extract_mapping_string(materials, enum_MATERIAL_ID)
    accessor_string += item.getter(enum_MATERIAL_ID)
    accessor_string += item.getter_string(enum_MATERIAL_ID)

In [455]:


def register_enum(s: pd.Series, singular: str, plural: str, axe: str, getter_name: str):
    main_string = (
        f'const enum {singular} ' '{\n'
        f'{s.apply(enum_name).drop_duplicates(keep="first").apply(comma).apply(new_line).apply(tab).sum()}'
        '}\n'
    )
    array_string = f'    static {plural}: {singular}[] = [{s.apply(enum_name).drop_duplicates(keep="first").apply(collection_name(singular)).apply(comma).apply(space).sum()}]\n'
    getter_string = accessor(f'{enum_name(singular)}_STRING', getter_name + "_string")
    getter_enum = accessor(f'{enum_name(plural)}', getter_name)
    return main_string, array_string, getter_string, getter_enum

In [456]:
# enum, array, getter_string, getter = register_enum(materials["tag"], ENUM_MATERIAL, ENUM_MATERIAL_PLURAL, "tag", "id")
# configuration_string += array
# main_string += enum
# accessor_string += getter + getter_string

# enum, array, getter_string, getter = register_enum(materials["category"], ENUM_CATEGORY, ENUM_CATEGORY_PLURAL, "category", "category")
# configuration_string += array
# main_string += enum
# accessor_string += getter + getter_string


# configuration_string += f'    static {enum_name(ENUM_MATERIAL)}_STRING : Record<{ENUM_MATERIAL}, string> = [{materials["tag"].apply(str).apply(quotes).apply(comma).apply(space).sum()}]\n'
# configuration_string += f'    static {enum_name(ENUM_CATEGORY)}_STRING : Record<{ENUM_MATERIAL}, string> = [{materials["category"].apply(str).apply(quotes).apply(comma).apply(space).sum()}]'

### Data

In [457]:
def number_series_to_array(s: pd.Series, id_enum: DataEnum, axis_name) -> str:
    return f'    static {id_enum.name_singular}_{enum_name(axis_name)} : Record<{id_enum.name_singular}, number> = [{s.apply(str).apply(comma).apply(space).sum()}]'

def string_series_to_array(s: pd.Series, id_enum: DataEnum, axis_name) -> str:
    return f'    static {id_enum.name_singular}_{enum_name(axis_name)} : Record<{id_enum.name_singular}, string> = [{s.apply(quotes).apply(comma).apply(space).sum()}]'

In [458]:
for item in number_columns:
    configuration_string = "\n".join([configuration_string, number_series_to_array(materials[item], enum_MATERIAL_ID, item)])
    accessor_string = "".join([accessor_string, accessor(f'{enum_MATERIAL_ID.name_singular}_{enum_name(item)}', item)])

for item in string_columns:
    configuration_string = "\n".join([configuration_string, string_series_to_array(materials[item], enum_MATERIAL_ID, item)])
    accessor_string = "".join([accessor_string, accessor(f'{enum_MATERIAL_ID.name_singular}_{enum_name(item)}', item)])

### Fat material

In [459]:
main_string = "\n".join([main_string, """


export interface MaterialData {
    readonly id: MATERIAL
    readonly category: MATERIAL_CATEGORY
    readonly id_string: string
    readonly category_string: string
    readonly cutting_power: number
    readonly density: number
    readonly cutting_protection: number
    readonly blunt_protection: number
    readonly penentration_protection: number
    readonly magic: number
}
"""])

In [460]:
configuration_string = (
    f'export class MaterialConfiguration {{\n'
    f'{configuration_string}\n'
    # f'    get_instance() {{\n'
    # f'        if (MaterialConfiguration._instance) return MaterialConfiguration._instance;\n'
    # f'        MaterialConfiguration._instance = new MaterialConfiguration()\n'
    # f'    }}\n'
    f'}}\n'
)

In [461]:


accessor_string = (
    f'\n'
    f'export class Material implements MaterialData {{\n'
    f'    private _id: MATERIAL\n'
    f'    constructor(id: MATERIAL) {{\n'
    f'        if (!(id in MaterialConfiguration.MATERIAL_MATERIAL_STRING)) {{\n'
    f'            throw new Error(`Invalid material id: ${{id}}`);\n'
    f'        }}\n'
    f'        this._id = id\n'
    f'    }}\n'
    f'    static from_string(id: string) {{\n'
    f'        if (!(id in MaterialConfiguration.MATERIALS_FROM_STRING)) {{\n'
    f'            throw new Error(`Invalid material id: ${{id}}`);\n'
    f'        }}\n'
    f'        return new Material(MaterialConfiguration.MATERIALS_FROM_STRING[id])\n'
    f'    }}\n'
    f'{accessor_string}'
    f'}}\n'
)

print(main_string +  configuration_string + accessor_string, file=open("src/materials.ts", "w"))